In [99]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [100]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [101]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())
print(data.collect().limit(5))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None
shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 28.121319 ┆ -100.386321 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:04:19.278943 ┆ 28.121319 ┆ -100.386321 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:30:00        ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:31:27.308903 ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:33:09.603331 ┆ 28.131229 ┆ 

## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [102]:
# Asegurarte de que todos los valores sean cadenas
df = data.with_columns(
    pl.col("timestamp").cast(pl.Utf8).alias("timestamp")  # Forzar a que sean cadenas
)

# Manejar timestamps sin microsegundos
df = df.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")  # Agregar microsegundos si no están presentes
    .alias("timestamp")
)

# Convertir timestamp de string a datetime
df = df.with_columns(
    pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.f").alias("timestamp")
)

# Mostrar los primeros resultados
print(df.collect().head(5))
print(type(df))

shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[μs]               ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 28.121319 ┆ -100.386321 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:04:19.278943 ┆ 28.121319 ┆ -100.386321 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:30:00        ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:31:27.308903 ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:33:09.603331 ┆ 28.131229 ┆ -100.386086 ┆ low_fuel      │
└────────┴────────────────────────────┴───────────┴─────────────┴───────────────┘
<class 'polars.lazyframe.frame.LazyFrame'>


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [103]:
# Se hace una copia para observar los resultados
df1 = df.clone()

# Ordenar los datos por car_id y timestamp
df1 = df1.sort(["car_id", "timestamp"])

# Calcular la diferencia de tiempo entre notificaciones del mismo vehículo
df1 = df1.with_columns(
    (pl.col("timestamp") - pl.col("timestamp").shift(1)).alias("notification_time")  # Diferencia entre timestamps
)

# Asegurar que la diferencia solo se calcula dentro de cada car_id
df1 = df1.with_columns(
    pl.when(pl.col("car_id") != pl.col("car_id").shift(1))
    .then(None)
    .otherwise(pl.col("notification_time"))
    .alias("notification_time")
)

# Recolectar datos 
df1 = df1.collect()  # Convierte el LazyFrame en un DataFrame materializado

# Mostrar los primeros resultados
print(df1.head(10))
print(type(df1))

shape: (10, 6)
┌────────┬─────────────────────┬───────────┬─────────────┬───────────────┬───────────────────┐
│ car_id ┆ timestamp           ┆ latitude  ┆ longitude   ┆ notification  ┆ notification_time │
│ ---    ┆ ---                 ┆ ---       ┆ ---         ┆ ---           ┆ ---               │
│ str    ┆ datetime[μs]        ┆ f64       ┆ f64         ┆ str           ┆ duration[μs]      │
╞════════╪═════════════════════╪═══════════╪═════════════╪═══════════════╪═══════════════════╡
│ car_0  ┆ 2023-01-01 00:00:00 ┆ 28.121319 ┆ -100.386321 ┆ low_fuel      ┆ null              │
│ car_0  ┆ 2023-01-01          ┆ 28.121319 ┆ -100.386321 ┆ engine_check  ┆ 4m 19s 278943µs   │
│        ┆ 00:04:19.278943     ┆           ┆             ┆               ┆                   │
│ car_0  ┆ 2023-01-01 00:30:00 ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure ┆ 25m 40s 721057µs  │
│ car_0  ┆ 2023-01-01          ┆ 28.131229 ┆ -100.386086 ┆ low_fuel      ┆ 6s 961005µs       │
│        ┆ 00:30:06.961005     ┆   

Después, podemos verificar que, los primeros coches (por `car_id`) según el timestamp, no tengan una diferencia de tiempo de notificación. Esto se debe a que no se tiene con que comparar para calcular la diferencia.

In [104]:
# Asegurarse de que los datos estén en un LazyFrame
df1 = df1.lazy()

# Agrupar por car_id y seleccionar la primera fila de cada grupo según el timestamp
first_rows = df1.group_by("car_id").agg([
    pl.col("timestamp").first().alias("first_timestamp"),        # Primera marca de tiempo
    pl.col("latitude").first().alias("first_latitude"),          # Primera latitud
    pl.col("longitude").first().alias("first_longitude"),        # Primera longitud
    pl.col("notification").first().alias("first_notification"),  # Primera notificación
    pl.col("notification_time").first().alias("first_notification_time")  # Primera notification_time
])

# Ordenar por car_id después de recolectar
result = first_rows.sort("car_id").collect()

# Mostrar los resultados
print(result.head(10))
print(type(result))

shape: (10, 6)
┌─────────┬─────────────────┬────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ car_id  ┆ first_timestamp ┆ first_latitude ┆ first_longitude ┆ first_notificat ┆ first_notificat │
│ ---     ┆ ---             ┆ ---            ┆ ---             ┆ ion             ┆ ion_time        │
│ str     ┆ datetime[μs]    ┆ f64            ┆ f64             ┆ ---             ┆ ---             │
│         ┆                 ┆                ┆                 ┆ str             ┆ duration[μs]    │
╞═════════╪═════════════════╪════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ car_0   ┆ 2023-01-01      ┆ 28.121319      ┆ -100.386321     ┆ low_fuel        ┆ null            │
│         ┆ 00:00:00        ┆                ┆                 ┆                 ┆                 │
│ car_1   ┆ 2023-01-01      ┆ 16.267287      ┆ -108.501117     ┆ tire_pressure   ┆ null            │
│         ┆ 00:00:00        ┆                ┆                 ┆            

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [105]:
df2 = df1.clone()
df2 = df2.lazy()

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    if lat1 is None or lon1 is None or lat2 is None or lon2 is None:
        return None
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])  # Convertir a radianes
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # Distancia en kilómetros

# Realizar el cálculo de la distancia, ordenado por vehículo y por tiempo
df2 = (
    df2.sort(["car_id", "timestamp"])  # Ordenar por vehículo y tiempo
    .with_columns(
        pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),  # Latitud previa por grupo
        pl.col("longitude").shift(1).over("car_id").alias("prev_longitude")  # Longitud previa por grupo
    )
    .with_columns(
        pl.struct(["latitude", "longitude", "prev_latitude", "prev_longitude"]).map_elements(
            lambda row: haversine(
                row["latitude"], row["longitude"], row["prev_latitude"], row["prev_longitude"]
            ) if row["prev_latitude"] is not None else None, return_dtype=pl.Float64
        ).alias("distance")
    )
    .drop(["prev_latitude", "prev_longitude"])  # Limpiar columnas temporales
)

# Recolectar datos para mostrar resultados
df2 = df2.collect()
print(df2.head(5))

shape: (5, 7)
┌────────┬──────────────────┬───────────┬─────────────┬───────────────┬─────────────────┬──────────┐
│ car_id ┆ timestamp        ┆ latitude  ┆ longitude   ┆ notification  ┆ notification_ti ┆ distance │
│ ---    ┆ ---              ┆ ---       ┆ ---         ┆ ---           ┆ me              ┆ ---      │
│ str    ┆ datetime[μs]     ┆ f64       ┆ f64         ┆ str           ┆ ---             ┆ f64      │
│        ┆                  ┆           ┆             ┆               ┆ duration[μs]    ┆          │
╞════════╪══════════════════╪═══════════╪═════════════╪═══════════════╪═════════════════╪══════════╡
│ car_0  ┆ 2023-01-01       ┆ 28.121319 ┆ -100.386321 ┆ low_fuel      ┆ null            ┆ null     │
│        ┆ 00:00:00         ┆           ┆             ┆               ┆                 ┆          │
│ car_0  ┆ 2023-01-01       ┆ 28.121319 ┆ -100.386321 ┆ engine_check  ┆ 4m 19s 278943µs ┆ 0.0      │
│        ┆ 00:04:19.278943  ┆           ┆             ┆               ┆      

Ya hecho el código, podemos verificar si se realizan correctamente las distancias, ya que en los primeros registros el vehículo no se está moviendo.

In [106]:
# Filtrar las filas donde la distancia es mayor a 0
filtered_df = df2.filter(pl.col("distance") > 0)

# Mostrar dos filas adjuntas
print(filtered_df.limit(2))

shape: (2, 7)
┌────────┬──────────────┬───────────┬─────────────┬───────────────┬─────────────────────┬──────────┐
│ car_id ┆ timestamp    ┆ latitude  ┆ longitude   ┆ notification  ┆ notification_time   ┆ distance │
│ ---    ┆ ---          ┆ ---       ┆ ---         ┆ ---           ┆ ---                 ┆ ---      │
│ str    ┆ datetime[μs] ┆ f64       ┆ f64         ┆ str           ┆ duration[μs]        ┆ f64      │
╞════════╪══════════════╪═══════════╪═════════════╪═══════════════╪═════════════════════╪══════════╡
│ car_0  ┆ 2023-01-01   ┆ 28.131229 ┆ -100.386086 ┆ tire_pressure ┆ 25m 40s 721057µs    ┆ 1.102235 │
│        ┆ 00:30:00     ┆           ┆             ┆               ┆                     ┆          │
│ car_0  ┆ 2023-01-01   ┆ 28.133137 ┆ -100.395682 ┆ low_fuel      ┆ 3m 52s 773328µs     ┆ 0.964643 │
│        ┆ 01:00:00     ┆           ┆             ┆               ┆                     ┆          │
└────────┴──────────────┴───────────┴─────────────┴───────────────┴──────────

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [107]:
df3 = df2.clone()
df3 = df3.lazy()

# Crear intervalos dinámicos por tiempo para cada car_id
x = '45m'  # Intervalo de 10 minutos
interval = x  

result = (
    df3
    .group_by_dynamic(
        index_column="timestamp",
        every=interval,    # Cada cuánto comienza un nuevo intervalo
        period=interval,   # Duración del intervalo
        closed="right",    # Cerrar los intervalos hacia atrás
        group_by="car_id"        # Agrupación por car_id
    )
    .agg([
        # Métricas para notification_time
        pl.col("notification_time").mean().alias("notification_time_mean"),
        pl.col("notification_time").median().alias("notification_time_median"),
        pl.col("notification_time").var().alias("notification_time_variance"),
        pl.col("notification_time").max().alias("notification_time_max"),
        pl.col("notification_time").min().alias("notification_time_min"),
        
        # Métricas para distance
        pl.col("distance").mean().alias("distance_mean"),
        pl.col("distance").median().alias("distance_median"),
        pl.col("distance").var().alias("distance_variance"),
        pl.col("distance").max().alias("distance_max"),
        pl.col("distance").min().alias("distance_min"),
    ])
)

# Recolectar datos y mostrar resultados
result = result.collect()
print(result.head(10))


shape: (10, 12)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ timestamp  ┆ notificati ┆ notificat ┆ … ┆ distance_ ┆ distance_ ┆ distance_ ┆ distance_ │
│ ---    ┆ ---        ┆ on_time_me ┆ ion_time_ ┆   ┆ median    ┆ variance  ┆ max       ┆ min       │
│ str    ┆ datetime[μ ┆ an         ┆ median    ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ s]         ┆ ---        ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│        ┆            ┆ duration[μ ┆ duration[ ┆   ┆           ┆           ┆           ┆           │
│        ┆            ┆ s]         ┆ μs]       ┆   ┆           ┆           ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2022-12-31 ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆ 23:15:00   ┆            ┆           ┆   ┆           ┆           